# Part 1 - Introduction to images, meshes and perspective projection

We're going to make use of quite a few different libraries in our lil crash course. We'll also need to grab some extra assets such as images and 3D models.

Today we'll be using numpy, matplotlib, OpenCV and a library for reading 3D meshes in PLY format.

If you haven't already, install the required modules using the requirements.txt file, and ensure this notebook is using the right virtual environment

The following code block downloads an image and mesh for use later.

In [ ]:
# Download an image
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/6/6f/York_Minster_from_M%26S.JPG/440px-York_Minster_from_M%26S.JPG -P ./assets/

# Download a mesh in ply format
!wget https://people.sc.fsu.edu/~jburkardt/data/ply/shark.ply -P ./assets/

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from plyfile import PlyData, PlyElement

Now let's load an image using OpenCV and see what it looks like using matplotlib.

In [ ]:
# Load image using OpenCV
img = cv2.imread('assets/440px-York_Minster_from_M&S.JPG') 
print('Image dimensions:',img.shape)
print('Image datatype:',img.dtype)

plt.imshow(img)

So far, so easy. We can see that cv2.imread has returned a numpy array of size 330 pixels high, 440 pixels wide and with three colour channels. 
We can also see that the values in the image are 8 bit unsigned integers. 
This is the most common way to store an image and means that for each pixel, you will have three numbers in the range 0..255 representing the intensity for each colour channel.

However, the astute among you may have noticed something off about the colour of the image we've displayed...
OpenCV decided to use the convention that colour channels are ordered: blue, green, red (BGR) rather than the (now) much more common convention: red, green, blue (RGB). 
If you're using any other library to process colour images, you probably need to convert to RGB. 
Let's do that and then display the image again using matplotlib.

In [ ]:
# Convert BGR to RGB
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# Display image
plt.imshow(img_rgb)

For clarity, we can now index values in the image as `img_rgb[row,column,colour_channel]` so that, for example, the red, green and blue values at the top left pixel are:

In [ ]:
# Extract the individual pixel values from the 3D array
print('Red value at pixel (0,0):',img_rgb[0,0,0])
print('Green value at pixel (0,0):',img_rgb[0,0,1])
print('Blue value at pixel (0,0):',img_rgb[0,0,2])

Representing image data as integers in the range 0..255 is less convenient when we want to start to manipulate the values in some way. 
The use of integers will introduce rounding errors that will accumulate over multiple operations. 
Therefore, we often want to convert image data to floating point values in the range 0..1 with 0 representing no light and 1 the maximum that can be recorded/displayed:

In [ ]:
img_float = img_rgb.astype('float32')/255

Now, we can apply arbitrary maths to the pixel values in our image. For example, we can multiply by an arbitrary factor to change the brightness of the image

In [ ]:
plt.imshow(img_float * 1.5)

The warning message is interesting here. 
Because we multiplied the image by 1.5, we now have values that are >1. 
`imshow` "clips" these values to 1, i.e. it uses `min(img_rgb,1.0)`. 
This simulates "saturation" in a camera. 
It can't record anymore light than 1. 
As each colour channel reaches 1, you end up with `(1.0,1.0,1.0)` for the three colour channels which means white. 
Hence, the blue sky has turned white.

## Task 1: Manipulating Images [6]

Let's do some more experimentation! Try scaling by different values and observe what happens; how about if you scale each colour channel by a different value? What does the image look like if, at each pixel, the value is the same for the three colour channels? (e.g. you might copy the red channel into green and blue, or take the average over the three channels). Can you think why it looks like this? What does it look like if you set two of the colour channels to zero?

Take a look through this [OpenCV tutorial](https://docs.opencv.org/4.x/d3/df2/tutorial_py_basic_ops.html) which goes over some of the above plus a bit more.

Can you *threshold* the image? This means turning an image into a binary image. For example, you might want to threshold the red channel on 0.5. So values $<0.5$ get set to 0 and $\geq 0.5$ get set to 1. To display such an image with imshow, you might want to pass `cmap='gray'` to get the expected appearance.

Finally, try cropping the image by selecting specific ranges of pixels.

If you have other ideas, try them out in the final box!

In [ ]:
fig, axs = plt.subplots(2, 3)
fig.set_size_inches(12,6)

# Try scaling each channel differently to change the warmth of the image
img_warm = ...
img_cold = ...

# Make a greyscale image by copying or combining the channels
img_gray = ...

# Threshold the image so we get a black & white mask. e.g. try to mask out the sky
img_mask = ...

# Crop the image around the minster
img_crop = ...

# Free space! Be creative :)
img_free = ...

axs[0,0].imshow(img_warm)
axs[0,1].imshow(img_cold)
axs[0,2].imshow(img_gray)
axs[1,0].imshow(img_mask, cmap='gray')
axs[1,1].imshow(img_crop)
axs[1,2].imshow(img_free)

## Projecting and drawing meshes

Perspective projection is a transformation we use to translate 3D real-world coordinates into 2D pixel coordinates on our camera sensor.

Here we model a simple pinhole camera so we can see the transformations we need to describe:


![Perspective projection diagram](images/perspective_projection.png "Perspective projection")

_N.B: It is normal to model the "image plane" as being in front of the camera (i.e. between the camera and the object) rather than behind the focal point of the lens, as this makes some of the maths and reasoning easier, and means we don't have to invert the image etc._

In this image we have two coordinate systems we want to convert between - the real world coordinates `(u,v,w)` and the image or pixel coordinates `(x,y)`

By taking our "optical center" as the center of our world coordinate system, it makes it easy to reason about these transformations.

We can show that the matrix for this transformation is as follows, where $f$ is the combined scale factor based on the focal distance and the size of the sensor

$$
\mathbf{K} = \begin{bmatrix} f & 0 & c_x \\ 0 & f & c_y \\ 0 & 0 & 1\end{bmatrix}
$$

In order to have more than one perspective on an object, we may want to move the camera around the world - or rather, the world around the camera...
If we want to change our view, we actually apply transformations first to the points we're looking at, _then_ apply the intrinsic camera properties matrix $\mathbf{K}$.


To do this we use a rotation matrix $\mathbf{R}$ and a translation vector $\mathbf{t}$ which we can compose together in a homogeneous transformation like so:

$$
\lambda  \begin{bmatrix} x \\ y \\ 1 \end{bmatrix} = 
\begin{bmatrix} \mathbf{K} & \mathbf{0} \end{bmatrix}
\begin{bmatrix} \mathbf{R} & \mathbf{t} \\ \mathbf{0}^T & 1\end{bmatrix}
\begin{bmatrix} u \\ v \\ w \\ 1 \end{bmatrix}
$$

Here, $\lambda$ represents the scalar required to convert back after the homogeneous transformation

We can actually simplify this further, and let's use $\mathbf{\tilde{x}}$ to represent pixel coordinates and $\mathbf{\tilde{w}}$ for world coordinates:

$$
\lambda \mathbf{\tilde{x}} = 
\mathbf{K}
\begin{bmatrix} \mathbf{R} & \mathbf{t}\end{bmatrix}
\mathbf{\tilde{w}}
$$

Because we can compose these matricies beforehand we operate on our world points, this means we can end up with a single linear transformation we can apply to a set of points that represent both the camera properties, and its relative position and orientation in the world.

## Task 2: Perspective Projection [5]

Now I would like you to compute the perspective projection of the vertices of the mesh into 2D (you can ignore the faces for now - we're just treating it like a point cloud).
I've given you the intrinsic and extrinsic camera parameters to perform a reasonable projection into an image that is 500 pixels wide and 326 pixels tall.

You need to write some code to apply the perspective projection transformation to our list of points `V` using the `K`, `R` and `t` matrices I've given.

_Hint: remember that you need to use homogeneous coordinates for the 3D vertices, do the matrix multiplications, then homogenise._

The following code block loads a 3D triangle mesh containing `nvertices` vertices and `nfaces` triangle faces. Vertices are stored in the 3 $\times$ `nvertices` numpy array `V`. So `V[:,0]` contains the 3D position of the first vertex. Faces are stored in the `nfaces` $\times$ 3 numpy array `F`. The three vertex indices for the first triangle are stored in `F[0,:]`. Putting those together, the 3D coordinates of the first vertex in the first triangle would be given by `V[:,F[0,:]]`.

In [ ]:
plydata = PlyData.read('assets/shark.ply')

nvertices = len(plydata['vertex'])
nfaces = len(plydata['face'])

# Copy vertex positions into numpy array
V = np.empty((3,nvertices))
V[0,:] = plydata['vertex']['x']
V[1,:] = plydata['vertex']['y']
V[2,:] = plydata['vertex']['z']

# Copy face data into numpy array
F = np.vstack(plydata['face'].data['vertex_indices'])

# Preview data
print('Vertex 0:',V[:,0])
print('Vertex 1:',V[:,1])
print('Vertex 2:',V[:,2])
print('Face 0:',F[0,:])

Face 0 here is the tuple `(0,1,2)`, which tells us that there is a face made by joining the vertexes 0, 1 and 2.
This also tells us the _orientation_ of the face - i.e. which side of the face is outwards.
We can use this information in graphics applications to avoid having to render the back of faces and various other optimizations.

In [ ]:
# These are some camera paramters I have provided so we can get a nice looking output for the file we're using
# We will fiddle with them later, but leave them as they are until you get the point cloud rendering working!

# K is the intrinsic parameters of the camera
# it is made up of the focal length/scale factor (2290)
# and the center of projection x,y (250, 163)
K = np.array([[2290.0,    0.0, 250.0],
              [   0.0, 2290.0, 163.0],
              [   0.0,    0.0,   1.0]])

# R and T are the extrinsic parameters of the camera
# together they represent the _pose_ of the camera
# R represents a rotation
R = np.array([[0.9063, 0.0000, 0.4226],
              [0.0000,-1.0000, 0.0000],
              [0.4226, 0.0000,-0.9063]])

# T represents a translation in world coordinates
t = np.array([[1.0350], [0.3861], [15.0140]])

In [ ]:
# Perform perspective projection
# your output u should be list of 2d points, resulting from transforming the verticies V

# Your code goes here:
# u = ...

# hint: @ operator is matrix multiplication, not *
# using * will perform element-wise multiplication

In [ ]:
# Plot projected 2D point cloud
fig, ax = plt.subplots()
fig.set_size_inches(6,4)
ax.scatter(u[0,:], u[1,:], s=1)
ax.axis('equal')
ax.set(xlim=(0, 550), ylim=(0, 326))
plt.gca().invert_yaxis()
plt.show()

print("Friend!")

## Task 3: Working with Triangles [5]

Next, I'd like you to draw our friendo here as a wireframe. In other words, you should draw a line in 2D for every triangle edge. To draw a line from `(x1,y1)` to `(x2,y2)` just use `ax.plot([x1,x2],[y1,y2])`. You might want to play with some parameters of `plot` to make the wireframe look nice. I've started you off with a loop over triangles. For each triangle you're going to draw three lines in 2D using the projected vertex positions `u[0,:]` and `u[1,:]` that you computed above.

In [ ]:
# Draw wireframe
fig, ax = plt.subplots()
for i in range(nfaces):
  # Draw the three triangle edges
  # Your code goes here:
  # ...
  
ax.axis('equal')
ax.set(xlim=(0, 550), ylim=(0, 326))
plt.gca().invert_yaxis()
plt.show()

The observant amongst you may have noticed that the above code is wasteful. Edges are shared between triangles so every edge will have been draw twice. There are algorithms for extracting a list of unique edges from a triangle list. If you are inspired, go and find one of those algorithms and implement it!

## Task 4: projection parameters [4]

Now that you have code for projecting a mesh to 2D and displaying it in 2D, try playing with some of the parameters. I carefully chose $\mathbf{K}$, $\mathbf{R}$ and $\mathbf{t}$ so that the model is in a nice pose and fits exactly in the image. 

First try playing with the focal length, i.e. elements $(1,1)$ and $(2,2)$ in $\mathbf{K}$ (remember that when we talk about element positions in matrices we give them as (row,column) indices and start from 1 even though our python code will use zero indexing). Try making it bigger and smaller. What happens? How about if the two focal length values are not equal? Next, what happens if you change $c_x$ and $c_y$? (elements $(1,3)$ and $(2,3)$ in $\mathbf{K}$).

Second try playing with the extrinsic parameters. Can you rotate the object around the vertical axis? (You'll need to apply a y axis rotation to your vertices). For extra challenge, can you find out how to do an animation in matplotlib and get the mesh to continuously rotate?

Finally, you might like to experiment with some other meshes. [This page](https://people.sc.fsu.edu/~jburkardt/data/ply/ply.html) has loads in PLY format.

Copy the code from above and leave your results below, commenting what you changed and why

## Total Mark [20]